##### Copyright 2020 Los autores de TensorFlow.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# El modelo secuencial

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/keras/sequential_model"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/keras-team/keras-io/blob/master/tf/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/keras-team/keras-io/blob/master/guides/sequential_model.py"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/keras-io/tf/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Descargar cuaderno</a></td>
</table>

## Preparar

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Cuando usar un modelo secuencial

Un modelo `Sequential` es apropiado para **una pila simple de capas** donde cada capa tiene **exactamente un tensor de entrada y un tensor de salida** .

Esquemáticamente, el siguiente modelo `Sequential` :

In [0]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

es equivalente a esta función:

In [0]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

A Sequential model is **not appropriate** when:

- Su modelo tiene múltiples entradas o múltiples salidas
- Cualquiera de sus capas tiene múltiples entradas o múltiples salidas
- Necesitas compartir capas
- Quiere una topología no lineal (por ejemplo, una conexión residual, un modelo de varias ramas)

## Creando un modelo secuencial

Puede crear un modelo secuencial pasando una lista de capas al constructor secuencial:

In [0]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

Sus capas son accesibles a través del atributo de `layers` :

In [0]:
model.layers

También puede crear un modelo secuencial de forma incremental a través del método `add()` :

In [0]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

Tenga en cuenta que también hay un método `pop()` correspondiente para eliminar capas: un modelo secuencial se comporta de manera muy similar a una lista de capas.

In [0]:
model.pop()
print(len(model.layers))  # 2

También tenga en cuenta que el constructor secuencial acepta un argumento de `name` , como cualquier capa o modelo en Keras. Esto es útil para anotar gráficos de TensorBoard con nombres semánticamente significativos.

In [0]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

## Especificar la forma de entrada de antemano

Generalmente, todas las capas de Keras necesitan conocer la forma de sus entradas para poder crear sus pesos. Entonces, cuando crea una capa como esta, inicialmente, no tiene pesos:

In [0]:
layer = layers.Dense(3)
layer.weights  # Empty

Crea sus pesos la primera vez que se llama en una entrada, ya que la forma de los pesos depende de la forma de las entradas:

In [0]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

Naturally, this also applies to Sequential models. When you instantiate a Sequential model without an input shape, it isn't "built": it has no weights (and calling `model.weights` results in an error stating just this). The weights are created when the model first sees some input data:

In [0]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Una vez que un modelo está "construido", puede llamar a su método `summary()` para mostrar su contenido:

In [0]:
model.summary()

Sin embargo, puede ser muy útil cuando se crea un modelo secuencial de forma incremental para poder mostrar el resumen del modelo hasta el momento, incluida la forma de salida actual. En este caso, debe comenzar su modelo pasando un objeto de `Input` a su modelo, para que conozca su forma de entrada desde el principio:

In [0]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Tenga en cuenta que el objeto de `Input` no se muestra como parte de `model.layers` , ya que no es una capa:

In [0]:
model.layers

Una alternativa simple es simplemente pasar un argumento `input_shape` a su primera capa:

In [0]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Los modelos creados con una forma de entrada predefinida como esta siempre tienen pesos (incluso antes de ver cualquier dato) y siempre tienen una forma de salida definida.

En general, es una práctica recomendada especificar siempre la forma de entrada de un modelo secuencial de antemano si sabe cuál es.

## Un flujo de trabajo de depuración común: `add()` + `summary()`

When building a new Sequential architecture, it's useful to incrementally stack layers with `add()` and frequently print model summaries. For instance, this enables you to monitor how a stack of `Conv2D` and `MaxPooling2D` layers is downsampling image feature maps:

In [0]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Muy práctico, ¿verdad?


## Qué hacer una vez que tenga un modelo

Una vez que la arquitectura de su modelo esté lista, querrá:

- Entrene su modelo, evalúelo y ejecute inferencias. Consulte nuestra [guía de capacitación y evaluación con los bucles integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate/)
- Guarde su modelo en el disco y restáurelo. Consulte nuestra [guía para serializar y guardar](https://www.tensorflow.org/guide/keras/save_and_serialize/) .
- Acelere el entrenamiento de modelos aprovechando múltiples GPU. Consulte nuestra [guía sobre formación distribuida y con varias GPU](distributed_training) .

## Extracción de características con un modelo secuencial

Una vez que se ha creado un modelo secuencial, se comporta como un [modelo de API funcional](https://www.tensorflow.org/guide/keras/functional/) . Esto significa que cada capa tiene un atributo de `input` y `output` . Estos atributos se pueden usar para hacer cosas interesantes, como crear rápidamente un modelo que extrae los resultados de todas las capas intermedias en un modelo secuencial:

In [0]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

Aquí hay un ejemplo similar que solo extrae características de una capa:

In [0]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

## Transferir el aprendizaje con un modelo secuencial

El aprendizaje por transferencia consiste en congelar las capas inferiores en un modelo y solo entrenar las capas superiores. Si no está familiarizado con él, asegúrese de leer nuestra [guía para transferir el aprendizaje](https://www.tensorflow.org/guide/keras/transfer_learning/) .

Aquí hay dos planos de aprendizaje por transferencia comunes que involucran modelos secuenciales.

First, let's say that you have a Sequential model, and you want to freeze all layers except the last one. In this case, you would simply iterate over `model.layers` and set `layer.trainable = False` on each layer, except the last one. Like this:

```python
modelo = keras.Sequential ([keras.Input (shape = (784)) capas.Denso (32, activación = 'relu'), capas.Denso (32, activación = 'relu'), capas.Denso (32, activación = 'relu'), layer.Dense (10),]) # Presumiblemente, primero querrá cargar pesos preentrenados. model.load_weights (...) # Congelar todas las capas excepto la última. for layer in model.layers [: - 1]: layer.trainable = False # Recompilar y entrenar (esto solo actualizará los pesos de la última capa). model.compile (...) model.fit (...)
```

Otro plan común es usar un modelo secuencial para apilar un modelo previamente entrenado y algunas capas de clasificación recién inicializadas. Me gusta esto:

```python
# Load a convolutional base with pre-trained weights base_model = keras.applications.Xception(     weights='imagenet',     include_top=False,     pooling='avg')  # Freeze the base model base_model.trainable = False  # Use a Sequential model to add a trainable classifier on top model = keras.Sequential([     base_model,     layers.Dense(1000), ])  # Compile & train model.compile(...) model.fit(...)
```

Si transfiere el aprendizaje, probablemente se encontrará utilizando con frecuencia estos dos patrones.

¡Eso es todo lo que necesita saber sobre los modelos secuenciales!

Para obtener más información sobre la construcción de modelos en Keras, consulte:

- [Guía de la API funcional](https://www.tensorflow.org/guide/keras/functional/)
- [Guía para crear nuevas capas y modelos mediante subclases](https://www.tensorflow.org/guide/keras/custom_layers_and_models/)